In [ ]:
import pandas as pd
import janitor as jn
import pymc3 as pm

%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
df = pd.read_csv('../data/finch_beaks_2012.csv')
df = (jn.DataFrame(df)
      .clean_names()
      .rename_column('blenth', 'beak_length')
      .rename_column('bdepth', 'beak_depth')
      .encode_categorical('species')
     )
df.groupby('species').size()

Ignore the "unknown" bird for now.

In [ ]:
df.dtypes

In [ ]:
fortis_filter = df['species'] == 'fortis'
scandens_filter = df['species'] == 'scandens'

In [ ]:
with pm.Model() as beak_depth_model:
    fortis_mean = pm.Normal('fortis_mean', mu=0, sd=100)
    scandens_mean = pm.Normal('scandens_mean', mu=0, sd=100)
    
    fortis_sd = pm.HalfCauchy('fortis_sd', beta=100)
    scandens_sd = pm.HalfCauchy('scandens_sd', beta=100)
    
    nu = pm.Exponential('nu', lam=1/29.) + 1
    
    like_fortis = pm.StudentT('like_fortis', mu=fortis_mean, sd=fortis_sd, nu=nu,
                              observed=df[fortis_filter]['beak_depth'])
    like_scandens = pm.StudentT('like_scandens', mu=scandens_mean, sd=scandens_sd, nu=nu,
                                 observed=df[scandens_filter]['beak_depth'])

In [ ]:
with beak_depth_model:
    trace = pm.sample(2000, tuning=1000)

In [ ]:
pm.forestplot(trace, varnames=['fortis_mean', 'scandens_mean'])

In [ ]:
pm.plot_posterior(trace, varnames=['fortis_mean', 'scandens_mean'])